Import relevant packages

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Lambda, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
import json
import numpy as np
import random
import os

tf.enable_eager_execution()
#tf.compat.v1.set_random_seed(18)

Set up data loading functions.

In [ ]:
tf.__version__

In [ ]:
Xt = []
Yt = []
Xv = []
Yv = []

rng = np.random.default_rng()

def load(name):
    with open(name) as json_file:
        data = json.load(json_file) 
        ctraintask = []
        cvaltask = []
        l = len(data['train'])
        i = 0
        for t in data['train']:
            i += 1
            cin = np.array(t['input'])
            cinr = np.full((30, 30), -1)
            cinr[:cin.shape[0], :cin.shape[1]] = cin
            cout = np.array(t['output'])
            coutr = np.full((30, 30), -1)
            coutr[:cout.shape[0], :cout.shape[1]] = cout
            cX = np.concatenate((cinr, coutr))
            if not i == l:
                ctraintask.append(cX)
                Yt.append(name)
            else:
                cvaltask.append(cX)
                Yv.append(name)
        Xt.append(np.array(ctraintask))
        Xv.append(np.array(cvaltask))



Load data.

In [ ]:
with os.scandir('training/') as entries:
    for entry in entries:
        load("training/"+entry.name)

with os.scandir('test/') as entries:
    for entry in entries:
        load("test/"+entry.name)

Xt = np.array(Xt)
Xv = np.array(Xv)
Yt = np.array(Yt)
Yv = np.array(Yv)

Define model.

In [ ]:
channels = 18
filters = 3

model = keras.Sequential([
    tf.keras.layers.DepthwiseConv2D((3, 3), strides=[1, 1], padding="same", depth_multiplier=filters, activation="ReLU")
    tf.keras.layers.Conv2D(128, 1, activation="ReLU")
    tf.keras.layers.Conv2D(channels, 1, activation=None, kernel_initializer=tf.zeros_initializer)
])

